In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import newyes24_dbio as ydb
import pandas as pd 

In [2]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌.
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.yes24.com/Product/Category/MonthWeekBestSeller") # 크롤링 할 웹사이트 주소
driver.find_element(By.CSS_SELECTOR, "option[value='120']").click() # 120개 보기가 클릭이 됨. 처음하자마자 120이 찍히도록 올려놓자.

In [3]:
driver.find_element(By.CSS_SELECTOR, "option[value='120']").click() # 120개 보기가 클릭이 됨.

In [4]:
all_books = [] #all_books에 page가 로딩이 돼서 모여있을 것임.
for page in range(1,4):
    time.sleep(8)
    page_html = driver.page_source
    soup = bs(page_html, "lxml") #page_html
    book_list = soup.select('#yesBestList > li')
    all_books.append(book_list)
    driver.find_element(By.CSS_SELECTOR, f"a[title='{page+1}']").click
driver.close()

In [5]:
len(all_books[0]) # 페이지에는 다 저장이 돼 있으니 for문을 돌려서 정보를 뽑으면 돼.

120

In [6]:
yy = 2023
mon = 8

In [7]:
# 8월 베스트셀러 1-3페이지 리스트
result = pd.DataFrame()
for book_list2 in all_books:
    temp = ydb.extract_bookinfo(yy, mon, book_list2)
    df = pd.DataFrame(temp, columns=['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', 'author', 'publisher', 'pub_date', 'price', 'n_reviews', 'review_link', 'rating', 'tags'])
    result = pd.concat([result, df])
# 8월 베스트셀러 1-3페이지 목록의 세부정보 링크로 세부정보 수집
detail_page_data = ydb.detail_page_info(result['detail_link']) #detail_link만 가져와서 []list에 담아준 것임.
result['book_id'] = result['detail_link'].apply(lambda x:x.split("/")[-1])#book_id라는 컬럼 네임 한번 더 만들기
final_result_df = pd.merge(result, detail_page_data, how='inner', on="book_id") #bood_id가 같은 애만 합치겠다.
display(final_result_df.head(2))
final_result_df.info()#최종적으로 몇개가 있는지 확인

,yy,mon,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,...,review_link,rating,tags,book_id,page,weight,size,category,book_intro,pub_book_intro
0,2023,8,[도서],,세이노의 가르침,,https://www.yes24.com/Product/Goods/117014613,[세이노(SayNo)],데이원,2023년 03월,...,https://www.yes24.com/Product/Goods/117014613?...,9.1,"[#성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서, #크레마클럽에있어요]",117014613,736쪽,"1,030g",153*224*40mm,"[성공학/경력관리, 국내도서, 처세술/삶의 자세, 자기계발]",\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 원대...,\n\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 ...
1,2023,8,[도서],,세이노의 가르침,,https://www.yes24.com/Product/Goods/117014613,[세이노(SayNo)],데이원,2023년 03월,...,https://www.yes24.com/Product/Goods/117014613?...,9.1,"[#성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서, #크레마클럽에있어요]",117014613,736쪽,"1,030g",153*224*40mm,"[성공학/경력관리, 국내도서, 처세술/삶의 자세, 자기계발]",\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 원대...,\n\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 ...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080 entries, 0 to 1079
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   yy              1080 non-null   int64 
 1   mon             1080 non-null   int64 
 2   title_h         1080 non-null   object
 3   title_f         1080 non-null   object
 4   title_m         1080 non-null   object
 5   title_e         1080 non-null   object
 6   detail_link     1080 non-null   object
 7   author          1080 non-null   object
 8   publisher       1080 non-null   object
 9   pub_date        1080 non-null   object
 10  price           1080 non-null   object
 11  n_reviews       1080 non-null   object
 12  review_link     1080 non-null   object
 13  rating          1080 non-null   object
 14  tags            1080 non-null   object
 15  book_id         1080 non-null   object
 16  page            1080 non-null   object
 17  weight          1080 non-null   object
 18  size    

mysql에 데이터 프레임 저장하기
* pip install pymysql - python에서 mysql에 접속할 때 필요
* pip install sqlalchemy - sql문을 간결하게 사용하기 위해서 사용 ORM

In [8]:
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import MySQLdb


In [9]:
for col in final_result_df.columns:
    final_result_df[col] = final_result_df[col].apply(str) # 모든 자료형 문자로 바꾸기

In [10]:
engine = create_engine("mysql+mysqldb://root:"+"1234"+ "@127.0.0.1:3306/yes24best", encoding='utf-8')
conn = engine.connect() # 데이터베이스에 접속
final_result_df.to_sql(name='best202308', con=engine, if_exists='append', index=False)
conn.close()